# Task 2 RDD

In [0]:
from pyspark.sql import SparkSession
from sparkmeasure import StageMetrics
from pyspark.sql import functions as F
from operator import add
import operator
from functools import reduce
from datetime import datetime
import pyspark.sql.types as types
from pyspark.sql.types import DateType, LongType
from pyspark.sql.functions import col, avg, min, max, round, count as _count


spark = SparkSession.builder.appName("task2").getOrCreate()

dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"
size = "small"
year = 2000

In [0]:
def clean_column_names(df):
  tempList = [] #Edit01
  for col in df.columns:
      new_name = col.strip()
      new_name = "".join(new_name.split())
      new_name = new_name.replace('.','') 
      tempList.append(new_name) 

  return df.toDF(*tempList) 

In [0]:
def avg(ls):
    return reduce(lambda a, b: float(a) + float(b), ls) / len(ls)
def convert_24_hr(flight):
    if flight[6] == "2400": flight[6] = "0000"
    if flight[6] == "2400": flight[8] = "0000"
    return flight
def compute_delay(time_departed, time_scheduled):
    
    return 0
    
def task_2_rdd(spark_session, flights_path, airlines_path, year):
    flights_rdd = spark.sparkContext.textFile(flights_path).map(lambda x: x.split(",")) 
    airlines_rdd = spark.sparkContext.textFile(airlines_path).map(lambda x: x.split(",")) 
    # airlines from USA
    airlines_rdd = airlines_rdd.filter(lambda x : x[2] == "United States")
    # Carrier code and name columns only
    airlines_rdd = airlines_rdd.map(lambda x: (x[0], x[1]))
    # Takes only rows with flight_date == year
    flights_rdd = flights_rdd.filter(lambda x: x[2].split("-")[0] == str(year))
    # Filter null actual_departure_time
    flights_rdd = flights_rdd.filter(lambda x: x[8]!="")
    # Change 24:00:00 to 00:00:00 time
    flights_rdd = flights_rdd.map(convert_24_hr)
    # Take only carrier_code, scheduled_depature_time, actual_departure_time
    flights_rdd = flights_rdd.map(lambda x: (x[0], x[6], x[8]))
    # Rdd of computed delay
    flights_rdd = flights_rdd.map(lambda x: (x[0], compute_delay(x[2], x[1])))
  
    # Take only "late" departures
    flights_rdd = flights_rdd.filter(lambda x: (x[1] > 0))
    
    flights_rdd_group = flights_rdd.groupByKey()
    flights_rdd_result = flights_rdd_group.map(lambda x: (x[0], (len(x[1]), avg(x[1]), min(x[1]), max(x[1]))))
    joined_rdd = flights_rdd_result.join(airlines_rdd)
    
    
    #joined_rdd.saveAsTextFile("task_2_rdd.txt")
    print(joined_rdd.collect().take(10))

    
    
task_2_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_{size}.csv", f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", year)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2661918993677406> in <module> 
 40 
 41 
 ---> 42 task_2_rdd ( spark , f"{dbfs_fileStore_prefix}/{prefix}_flights_{size}.csv" , f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv" , year ) 

 <command-2661918993677406> in task_2_rdd (spark_session, flights_path, airlines_path, year) 
 36 
 37 #joined_rdd.saveAsTextFile("task_2_rdd.txt") 
 ---> 38 print ( joined_rdd . collect ( ) . take ( 10 ) ) 
 39 
 40 

 AttributeError : 'list' object has no attribute 'take'

In [0]:
task_2_rdd(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_{size}.csv", f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", year)
joined_rdd.show()

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2661918993677408> in <module> 
 ----> 1 task_2_rdd ( spark , f"{dbfs_fileStore_prefix}/{prefix}_flights_{size}.csv" , f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv" , year ) 
 2 joined_rdd . show ( ) 

 <command-2661918993677406> in task_2_rdd (spark_session, flights_path, airlines_path, year) 
 36 
 37 #joined_rdd.saveAsTextFile("task_2_rdd.txt") 
 ---> 38 print ( joined_rdd . collect ( ) . take ( 10 ) ) 
 39 
 40 

 AttributeError : 'list' object has no attribute 'take'